In [497]:
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np
lone_pine = pd.read_csv(r'C:\Users\wells\OneDrive\Desktop\lone_pine.csv')
crsp_data = pd.read_csv('StockPriceMonthly.csv', encoding='utf8', low_memory=False)

# pre-processing

In [498]:
lone_pine['Ticker'] = lone_pine['Ticker'].astype('str')
def ab(aa):
    if '.' in aa:
        return aa.split('.')[0]
    if '/' in aa:
        return aa.split('/')[0]
    else:
        return aa
lone_pine['Ticker'] = lone_pine['Ticker'].apply(ab)

In [499]:
def abc(aa):
    aa = aa.split('/')
    year = aa[2]
    month = aa[0]
    day = aa[1]
    return year+month
lone_pine['Date'] = lone_pine['Date'].apply(abc)

In [500]:
def abcd(aa):
    return str(aa)[0:6]
crsp_data['date'] = crsp_data['date'].apply(abcd)

In [501]:
add_df = pd.DataFrame(columns=lone_pine.columns)
for index in tqdm_notebook(range(len(lone_pine))):
    each_data = lone_pine.loc[index]
    date = int(each_data['Date'])
    if date % 4 == 0:
        date = date - 12 + 100
    each_data['Date'] = str(date+1)
    each_data['Change'] = 0
    each_data['Change.1'] = 0
    add_df.loc[-1] = list(each_data)
    add_df = add_df.reset_index(drop=True)
    date = int(each_data['Date'])
    each_data['Date'] = str(date+1)
    each_data['Change'] = 0
    each_data['Change.1'] = 0
    add_df.loc[-1] = list(each_data)
    add_df = add_df.reset_index(drop=True)
lone_pine_a = pd.concat([lone_pine, add_df], ignore_index=True)

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\wells\AppData\Local\Continuu

In [502]:
def class_(df_col):
    if df_col == 'CL A':
        return 'A'
    elif df_col == 'CL B':
        return 'B'
    elif df_col == 'CL C':
        return 'C'
    elif df_col == 'CL D':
        return 'D'
    else:
        return np.nan
lone_pine_a['Class'] = lone_pine_a['Class'].apply(class_)

In [503]:
def division(df):
    if df['Shares'] == 0:
        return 0
    else:
        return df['Value'] / df['Shares']
lone_pine_a['price'] = lone_pine_a.apply(division, axis=1)

In [504]:
crsp_data.columns = ['PERMNO', 'Date', 'Ticker', 'COMNAM', 'Class', 'DIVAMT', 'PRC', 'VOL']

In [505]:
# def class_1(df_col):
#     if df_col == 'A':
#         return 'A'
#     elif df_col == 'B':
#         return 'B'
#     elif df_col == 'C':
#         return 'C'
#     elif df_col == 'D':
#         return 'D'
#     else:
#         return 0
# crsp_data['Class'] = crsp_data['Class'].apply(class_1)

In [506]:
def to_abs(df_col):
    return abs(df_col)
crsp_data_c = crsp_data[['Date', 'Ticker', 'PRC', 'Class']]
crsp_data_c = crsp_data_c[crsp_data_c['PRC'].notnull()].drop_duplicates(keep='first')
crsp_data_c['PRC'] = crsp_data_c['PRC'].apply(to_abs)

In [507]:
lone_pine_all = lone_pine_a.merge(crsp_data_c, on=['Date', 'Ticker'], how = 'left')

In [508]:
lone_pine_all.drop(['% Port', '% OS', 'Hist'], axis=1, inplace=True)

In [509]:
for name in list(set(list(lone_pine_all['Name']))):
    if len(lone_pine_a[lone_pine_a['Name']==name].reset_index(drop=True)) != len(lone_pine_all[lone_pine_all['Name']==name].reset_index(drop=True)):
        drop_df = lone_pine_all[lone_pine_all['Name']==name].loc[lone_pine_all['Class_x']!=lone_pine_all['Class_y']]
        lone_pine_all.drop(list(drop_df.index), inplace=True)

In [510]:
lone_pine_all = lone_pine_all.sort_values(by='Date').reset_index(drop=True)
df_list = []
for each in tqdm_notebook(lone_pine_all.groupby(by='Name')):
    name, df = each
    df.loc[:, 'last_price'] = df['price'].shift()
    df.loc[:, 'last_PRC'] = df['PRC'].shift()
    df_list.append(df)

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [511]:
lone_pine_all = pd.concat(df_list)

In [512]:
error = lone_pine_all[lone_pine_all['PRC'].isnull()][['Name', 'Ticker', 'PRC', 'Date']].drop_duplicates(keep='first', inplace=False)

In [514]:
test = lone_pine_all
new_error = []
for index in tqdm_notebook(error.index):
    each = error.loc[index]
    df = crsp_data[crsp_data['Ticker']==each['Ticker']]
    try:
        if int(each['Date']) > int(list(df['Date'])[-1]):
            test = test.drop(index)
    except:
        new_error.append(list(each))
lone_pine_all = test

In [519]:
pd.DataFrame(new_error).to_csv('error.csv', index=False)

# revenue

In [536]:
lone_pine_all_t = lone_pine_all
for index in tqdm_notebook(lone_pine_all.index):
    each_data = lone_pine_all.loc[index]
    if each_data['Shares'] == 0 and each_data['Change'] == 0 and each_data['Value'] == 0 and each_data['Change.1'] == 0:
        lone_pine_all_t.drop(index, inplace=True)
lone_pine_all = lone_pine_all_t

In [571]:
def prc_price(df):
    if np.isnan(df['PRC']):
        return df['price']
    else:
        return df['PRC']
lone_pine_all['PRC'] = lone_pine_all.apply(prc_price, axis=1)

In [574]:
def prc_price(df):
    if np.isnan(df['last_PRC']):
        return df['last_price']
    else:
        return df['last_PRC']
lone_pine_all['last_PRC'] = lone_pine_all.apply(prc_price, axis=1)

In [577]:
lone_pine_all.drop(['Class_x', 'price', 'last_price', 'price'], axis=1, inplace=True)
lone_pine_all.columns = ['Name', 'Ticker', 'Shares', 'Shares_Change', 'Value', 'Value_Change', 'Date', 'Price',
                         'Class', 'Last_Price']

In [1088]:
new_share = lone_pine_all[lone_pine_all['Shares']==lone_pine_all['Shares_Change']]
new_share['New_Value'] = new_share['Shares'] * new_share['Price']

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1089]:
new_share.to_csv('new_share.csv', index=False)

In [1090]:
sell_share = lone_pine_all[lone_pine_all['Shares']==0]
sell_share['New_Value'] = sell_share['Shares_Change'] * sell_share['Price']

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1091]:
sell_share.to_csv('sell_share.csv', index=False)

In [1092]:
more_share = lone_pine_all[lone_pine_all['Shares_Change']>0].loc[lone_pine_all['Shares']!=lone_pine_all['Shares_Change']]
more_share['New_Value'] = more_share['Shares'] * more_share['Price']

In [1093]:
more_share.to_csv('more_share.csv', index=False)

In [1094]:
less_share = lone_pine_all[lone_pine_all['Shares_Change']<0].loc[lone_pine_all['Shares']!=0]
less_share['New_Value'] = (less_share['Shares'] + abs(less_share['Shares_Change'])) * less_share['Price']

In [1095]:
less_share.to_csv('less_share.csv', index=False)

In [1105]:
same_share = lone_pine_all[lone_pine_all['Shares_Change']==0]
same_share['New_Value'] = same_share['Shares'] * same_share['Price']

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1106]:
same_share.to_csv('same_share.csv', index=False)

In [1122]:
total_asset = []
for date in list(set(lone_pine_all['Date'])):
    new_share_d = new_share[new_share['Date']==date]
    sell_share_d = sell_share[sell_share['Date']==date]
    more_share_d = more_share[more_share['Date']==date]
    less_share_d = less_share[less_share['Date']==date]
    same_share_d = same_share[same_share['Date']==date]
    total_asset.append(sum(new_share_d['New_Value'])+abs(sum(sell_share_d['New_Value']))+sum(more_share_d['New_Value'])+sum(less_share_d['New_Value'])+sum(same_share_d['New_Value']))

In [1149]:
revenue = []
for date in list(set(lone_pine_all['Date'])):
    second = lone_pine_all[lone_pine_all['Date']==date]

    sell_shares = second[second['Shares']==0]
    more_shares = second[second['Shares_Change']>0].loc[second['Shares']!=second['Shares_Change']]
    less_shares = second[second['Shares_Change']<0].loc[second['Shares']!=0]
    same_shares = second[second['Shares_Change']==0]

    sell_shares['diff'] = abs(sell_shares['Shares_Change']) * (sell_shares['Price'] - sell_shares['Last_Price'])
    more_shares['diff'] = (more_shares['Shares'] - more_shares['Shares_Change']) * (more_shares['Price'] - more_shares['Last_Price'])
    less_shares['diff'] = (less_shares['Shares'] + abs(less_shares['Shares_Change'])) * (less_shares['Price'] - less_shares['Last_Price'])
    same_shares['diff'] = same_shares['Shares'] * (same_shares['Price'] - same_shares['Last_Price'])
    
    sell_shares = sell_shares[sell_shares['diff'].notnull()]
    more_shares = more_shares[more_shares['diff'].notnull()]
    less_shares = less_shares[less_shares['diff'].notnull()]
    same_shares = same_shares[same_shares['diff'].notnull()]
    
    revenue.append(sum(sell_shares['diff']) + sum(more_shares['diff']) + sum(less_shares['diff']) + sum(same_shares['diff']))

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [1150]:
revenue_df = pd.DataFrame({'date':list(set(lone_pine_all['Date'])),
                          'asset':total_asset,
                          'revenue':revenue})

In [1151]:
revenue_df['return_rate'] = round(revenue_df['revenue'] / revenue_df['asset'] * 100, 3)

In [1153]:
fama = pd.read_csv('fama.csv')
fama['date'] = fama['date'].astype('str')

fama_3 = fama.merge(revenue_df[['date', 'return_rate']], on='date', how='left')

fama_3 = fama_3[fama_3['return_rate'].notnull()]

fama_3['return_rate'] = fama_3['return_rate'] - fama_3['RF']

In [1154]:
fama = pd.read_csv('fama5.csv')
fama['date'] = fama['date'].astype('str')

fama_5 = fama.merge(revenue_df[['date', 'return_rate']], on='date', how='left')

fama_5 = fama_5[fama_5['return_rate'].notnull()]

fama_5['return_rate'] = fama_5['return_rate'] - fama_5['RF']

# lasso

In [1155]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [1156]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1157]:
alpha_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'train_R2', 'R2', 'e', 'coeff_used'])
alpha = 5
while alpha > 0.001:
    lasso = Lasso(alpha)
    lasso.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = lasso.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = lasso.predict(x_test_std)
    score = r2_score(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(lasso.coef_!=0)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    alpha_df.loc[-1] = [alpha] + list(lasso.coef_) + [score_train, score, e, coeff_used]
    alpha_df = alpha_df.reset_index(drop=True)
    alpha = alpha - 0.001

In [1158]:
alpha_df

,alpha,coef1,coef2,coef3,train_R2,R2,e,coeff_used
0,5.000,17.932967,-30.761090,-0.000000,0.018953,-44.057758,1414.453516,2.0
1,4.999,17.934427,-30.762551,-0.000000,0.018953,-44.060232,1414.531191,2.0
2,4.998,17.935888,-30.764012,-0.000000,0.018954,-44.062706,1414.608870,2.0
3,4.997,17.937349,-30.765472,-0.000000,0.018954,-44.065181,1414.686555,2.0
4,4.996,17.938810,-30.766933,-0.000000,0.018955,-44.067656,1414.764243,2.0
5,4.995,17.940271,-30.768394,-0.000000,0.018955,-44.070131,1414.841937,2.0
6,4.994,17.941732,-30.769855,-0.000000,0.018955,-44.072606,1414.919635,2.0
7,4.993,17.943192,-30.771316,-0.000000,0.018956,-44.075081,1414.997338,2.0
8,4.992,17.944653,-30.772777,-0.000000,0.018956,-44.077557,1415.075046,2.0
9,4.991,17.946114,-30.774237,-0.000000,0.018956,-44.080032,1415.152759,2.0


In [802]:
def pretty_print_linear(coefs, names = None, sort = False):  
    if names == None:  
        names = ["X%s" % x for x in range(len(coefs))]  
    lst = zip(coefs, names)  
    if sort:  
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))  
    return " + ".join("%s * %s" % (round(coef, 3), name)  
                                   for coef, name in lst)  
 
 
 
print('Y = '+ pretty_print_linear(lasso.coef_))

Y = -5.249 * X0 + 21.391 * X1 + 15.716 * X2


fama5

In [1159]:
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']]
y = fama_5[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1160]:
alpha_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'coef4', 'coef5', 'train_R2', 'R2', 'e', 'coeff_used'])
alpha = 5
while alpha > 0.001:
    lasso = Lasso(alpha)
    lasso.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = lasso.predict(x_train_std)
    score_train=r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = lasso.predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(lasso.coef_!=0)
    alpha_df.loc[-1] = [alpha] + list(lasso.coef_) + [score_train, score, e, coeff_used]
    alpha_df = alpha_df.reset_index(drop=True)
    alpha = alpha - 0.001

In [1161]:
alpha_df

,alpha,coef1,coef2,coef3,coef4,coef5,train_R2,R2,e,coeff_used
0,5.000,21.936123,-29.977548,-0.000000,7.113629,0.000000,0.019182,-34.253898,940.158761,3.0
1,4.999,21.937932,-29.979453,-0.000000,7.114943,0.000000,0.019183,-34.257979,940.267607,3.0
2,4.998,21.939742,-29.981358,-0.000000,7.116256,0.000000,0.019183,-34.262061,940.376465,3.0
3,4.997,21.941551,-29.983263,-0.000000,7.117570,0.000000,0.019184,-34.266144,940.485335,3.0
4,4.996,21.943361,-29.985168,-0.000000,7.118883,0.000000,0.019185,-34.270227,940.594217,3.0
5,4.995,21.945170,-29.987073,-0.000000,7.120196,0.000000,0.019185,-34.274310,940.703111,3.0
6,4.994,21.946980,-29.988978,-0.000000,7.121510,0.000000,0.019186,-34.278394,940.812017,3.0
7,4.993,21.948789,-29.990883,-0.000000,7.122823,0.000000,0.019187,-34.282478,940.920935,3.0
8,4.992,21.950599,-29.992788,-0.000000,7.124137,0.000000,0.019187,-34.286562,941.029864,3.0
9,4.991,21.952408,-29.994692,-0.000000,7.125450,0.000000,0.019188,-34.290648,941.138806,3.0


In [792]:
def pretty_print_linear(coefs, names = None, sort = False):  
    if names == None:  
        names = ["X%s" % x for x in range(len(coefs))]  
    lst = zip(coefs, names)  
    if sort:  
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))  
    return " + ".join("%s * %s" % (round(coef, 3), name)  
                                   for coef, name in lst)  
 
 
 
print('Y = '+ pretty_print_linear(lasso.coef_))

Y = 8.346 * X0 + -0.0 * X1 + 0.0 * X2 + 2.105 * X3 + -0.0 * X4


# Ridge

In [847]:
from sklearn.linear_model import Ridge

In [1162]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1163]:
alpha_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'train_R2', 'R2', 'e', 'coeff_used'])
alpha = 5
while alpha >= 0.001:
    rr = Ridge(alpha)
    rr.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = rr.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = rr.predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(rr.coef_!=0)
    alpha_df.loc[-1] = [alpha] + list(rr.coef_) + [score_train, score, e, coeff_used]
    alpha_df = alpha_df.reset_index(drop=True)
    alpha = alpha - 0.001

In [1164]:
alpha_df

,alpha,coef1,coef2,coef3,train_R2,R2,e,coeff_used
0,5.000,3.429015,0.791739,-0.509986,0.191419,-0.022219,222365.595553,3.0
1,4.999,3.429042,0.791735,-0.509993,0.191419,-0.022219,222365.593186,3.0
2,4.998,3.429069,0.791731,-0.510000,0.191419,-0.022219,222365.590820,3.0
3,4.997,3.429095,0.791727,-0.510008,0.191420,-0.022219,222365.588453,3.0
4,4.996,3.429122,0.791723,-0.510015,0.191420,-0.022219,222365.586086,3.0
5,4.995,3.429149,0.791719,-0.510022,0.191420,-0.022219,222365.583719,3.0
6,4.994,3.429176,0.791715,-0.510029,0.191420,-0.022219,222365.581352,3.0
7,4.993,3.429203,0.791711,-0.510036,0.191420,-0.022219,222365.578984,3.0
8,4.992,3.429230,0.791707,-0.510044,0.191420,-0.022219,222365.576617,3.0
9,4.991,3.429256,0.791703,-0.510051,0.191420,-0.022219,222365.574250,3.0


In [1165]:
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']]
y = fama_5[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1166]:
alpha_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'coef4', 'coef5', 'train_R2', 'R2', 'e', 'coeff_used'])
alpha = 5
while alpha >= 0.001:
    rr = Ridge(alpha)
    rr.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = rr.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = rr.predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(rr.coef_!=0)
    alpha_df.loc[-1] = [alpha] + list(rr.coef_) + [score_train, score, e, coeff_used]
    alpha_df = alpha_df.reset_index(drop=True)
    alpha = alpha - 0.001

In [1167]:
alpha_df

,alpha,coef1,coef2,coef3,coef4,coef5,train_R2,R2,e,coeff_used
0,5.000,27.590889,-42.262016,-2.614023,11.558587,5.762743,0.024335,-128.713317,2815.745687,5.0
1,4.999,27.591267,-42.262484,-2.614048,11.558759,5.762748,0.024335,-128.715366,2815.790184,5.0
2,4.998,27.591645,-42.262953,-2.614072,11.558931,5.762752,0.024335,-128.717416,2815.834683,5.0
3,4.997,27.592023,-42.263421,-2.614097,11.559103,5.762757,0.024335,-128.719466,2815.879184,5.0
4,4.996,27.592401,-42.263890,-2.614121,11.559275,5.762761,0.024335,-128.721516,2815.923686,5.0
5,4.995,27.592779,-42.264358,-2.614145,11.559447,5.762766,0.024335,-128.723567,2815.968189,5.0
6,4.994,27.593157,-42.264826,-2.614170,11.559619,5.762770,0.024335,-128.725617,2816.012694,5.0
7,4.993,27.593536,-42.265295,-2.614194,11.559792,5.762775,0.024335,-128.727667,2816.057201,5.0
8,4.992,27.593914,-42.265763,-2.614219,11.559964,5.762779,0.024335,-128.729717,2816.101709,5.0
9,4.991,27.594292,-42.266232,-2.614243,11.560136,5.762784,0.024335,-128.731768,2816.146218,5.0


# SVR

In [857]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve

In [1193]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1194]:
c_df = pd.DataFrame(columns=['c', 'coef1', 'coef2', 'coef3', 'train_R2', 'R2', 'e', 'coeff_used'])
c = 100
while c > 1:
    svr_lin = SVR(kernel='linear', C=c, gamma='auto')
    svr_lin.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = svr_lin.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = svr_lin.predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(svr_lin.coef_!=0)
    c_df.loc[-1] = [c] + list(svr_lin.coef_[0]) + [score_train, score, e, coeff_used]
    c_df = c_df.reset_index(drop=True)
    c = c - 1

In [1195]:
c_df

,c,coef1,coef2,coef3,train_R2,R2,e,coeff_used
0,100.0,3.523136,0.335669,-0.597087,-0.006007,0.711884,10.139220,3.0
1,99.0,3.523033,0.336046,-0.597204,-0.006008,0.711879,10.139382,3.0
2,98.0,3.523091,0.335748,-0.597189,-0.006008,0.711883,10.139237,3.0
3,97.0,3.523040,0.336082,-0.597186,-0.006008,0.711879,10.139379,3.0
4,96.0,3.523282,0.335800,-0.597206,-0.006008,0.711900,10.138649,3.0
5,95.0,3.523089,0.335874,-0.597124,-0.006008,0.711881,10.139310,3.0
6,94.0,3.523313,0.335728,-0.597213,-0.006008,0.711903,10.138549,3.0
7,93.0,3.523102,0.335814,-0.597108,-0.006008,0.711882,10.139293,3.0
8,92.0,3.523061,0.336002,-0.597174,-0.006008,0.711880,10.139333,3.0
9,91.0,3.523091,0.335705,-0.597198,-0.006008,0.711883,10.139227,3.0


In [1196]:
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']]
y = fama_5[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1172]:
c_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'coef4', 'coef5', 'train_R2', 'R2', 'e', 'coeff_used'])
c = 100
while c > 1:
    svr_lin = SVR(kernel='linear', C=c, gamma='auto')
    svr_lin.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = svr_lin.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = svr_lin.fit(x_train_std, y_train['return_rate'].values).predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(svr_lin.coef_!=0)
    c_df.loc[-1] = [c] + list(svr_lin.coef_[0]) + [score_train, score, e, coeff_used]
    c_df = c_df.reset_index(drop=True)
    c = c - 1

In [1173]:
c_df

,alpha,coef1,coef2,coef3,coef4,coef5,train_R2,R2,e,coeff_used
0,100.0,3.783283,0.440047,-0.110605,-1.094581,-0.144194,0.196043,-0.021755,222262.528455,5.0
1,99.0,3.783210,0.440173,-0.110598,-1.094467,-0.144245,0.196044,-0.021755,222262.528611,5.0
2,98.0,3.783277,0.439932,-0.110602,-1.094731,-0.144103,0.196043,-0.021755,222262.537746,5.0
3,97.0,3.783147,0.440286,-0.110507,-1.094452,-0.144264,0.196044,-0.021755,222262.539142,5.0
4,96.0,3.783273,0.440024,-0.110607,-1.094604,-0.144177,0.196043,-0.021755,222262.529466,5.0
5,95.0,3.783260,0.439992,-0.110633,-1.094611,-0.144160,0.196044,-0.021755,222262.528306,5.0
6,94.0,3.783265,0.440013,-0.110628,-1.094586,-0.144178,0.196044,-0.021755,222262.526429,5.0
7,93.0,3.783117,0.440253,-0.110497,-1.094573,-0.144193,0.196044,-0.021755,222262.552972,5.0
8,92.0,3.783267,0.439917,-0.110620,-1.094709,-0.144107,0.196043,-0.021755,222262.533617,5.0
9,91.0,3.783269,0.440008,-0.110633,-1.094584,-0.144180,0.196044,-0.021755,222262.525212,5.0


# LinearRegression

In [892]:
from sklearn.linear_model import LinearRegression

In [1174]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1175]:
lr = LinearRegression(alpha)
lr.fit(x_train_std, y_train['return_rate'].values)
y_predict_train = lr.fit(x_train_std, y_train['return_rate'].values).predict(x_train_std)
score_train = r2_score(y_train['return_rate'].values, y_predict_train)
y_predict = lr.fit(x_train_std, y_train['return_rate'].values).predict(x_test_std)
score=r2_score(y_test['return_rate'].values, y_predict)
e = mean_squared_error(y_test['return_rate'].values, y_predict)
coeff_used = np.sum(lr.coef_!=0)
print('score_train:',score_train)
print('score_prediction:',score)
print('e:',e)
# list(lr.coef_) + [score_train, score, e, coeff_used]

score_train: 0.01909437431279304
score_prediction: -67.99021215993032
e: 1623.261360714524


In [1176]:
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']]
y = fama_5[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1177]:
lr = LinearRegression(alpha)
lr.fit(x_train_std, y_train['return_rate'].values)
y_predict_train = lr.fit(x_train_std, y_train['return_rate'].values).predict(x_train_std)
score_train = r2_score(y_train['return_rate'].values, y_predict_train)
y_predict = lr.fit(x_train_std, y_train['return_rate'].values).predict(x_test_std)
score=r2_score(y_test['return_rate'].values, y_predict)
e = mean_squared_error(y_test['return_rate'].values, y_predict)
coeff_used = np.sum(lr.coef_!=0)
print('score_train:',score_train)
print('score_prediction:',score)
print('e:',e)
# list(lr.coef_) + [score, e, coeff_used]

score_train: 0.02261728631670623
score_prediction: -128.592496385564
e: 2237.0500870132537
